In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [2]:
features = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/features.npy"))
labels = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/labels.npy"))
#features = features.reshape(list(features.shape)+ [1])

print(features.shape)
print(labels.shape)


x_train, x_test, y_train, y_test = train_test_split(features,labels)


(4445, 432, 614, 3)
(4445, 2)


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(432,614,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 430, 612, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 215, 306, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 213, 304, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 106, 152, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1031168)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                65994816  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [4]:


model.compile(optimizer='adam',
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train model with new callback
fit = model.fit(x_train,
                y_train,
                epochs=80, 
                batch_size=batch_size, 
                callbacks=[cp_callback], 
                validation_data=(x_test,y_test))

Epoch 1/80
105/105 [==============================] - 24s 203ms/step - loss: 732531.8628 - mean_squared_error: 732531.8628 - val_loss: 0.4776 - val_mean_squared_error: 0.4776
Epoch 2/80
 55/105 [==============>...............] - ETA: 8s - loss: 0.4074 - mean_squared_error: 0.4074
Epoch 00002: saving model to training/cp-0002.ckpt
105/105 [==============================] - 21s 199ms/step - loss: 0.3655 - mean_squared_error: 0.3655 - val_loss: 0.1925 - val_mean_squared_error: 0.1925
Epoch 3/80
105/105 [==============================] - 20s 189ms/step - loss: 0.1710 - mean_squared_error: 0.1710 - val_loss: 0.1248 - val_mean_squared_error: 0.1248
Epoch 4/80
  5/105 [>.............................] - ETA: 17s - loss: 0.1443 - mean_squared_error: 0.1443
Epoch 00004: saving model to training/cp-0004.ckpt
105/105 [==============================] - 21s 198ms/step - loss: 0.1130 - mean_squared_error: 0.1130 - val_loss: 0.0925 - val_mean_squared_error: 0.0925
Epoch 5/80
 60/105 [================>

105/105 [==============================] - 21s 198ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0274 - val_mean_squared_error: 0.0274
Epoch 33/80
105/105 [==============================] - 20s 189ms/step - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 34/80
 55/105 [==============>...............] - ETA: 8s - loss: 0.0109 - mean_squared_error: 0.0109
Epoch 00034: saving model to training/cp-0034.ckpt
105/105 [==============================] - 21s 196ms/step - loss: 0.0107 - mean_squared_error: 0.0107 - val_loss: 0.0291 - val_mean_squared_error: 0.0291
Epoch 35/80
105/105 [==============================] - 20s 189ms/step - loss: 0.0105 - mean_squared_error: 0.0105 - val_loss: 0.0275 - val_mean_squared_error: 0.0275
Epoch 36/80
  5/105 [>.............................] - ETA: 17s - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 00036: saving model to training/cp-0036.ckpt
105/105 [==============================] - 

Epoch 64/80
105/105 [==============================] - ETA: 0s - loss: 0.0190 - mean_squared_error: 0.0190
Epoch 00064: saving model to training/cp-0064.ckpt
105/105 [==============================] - 21s 196ms/step - loss: 0.0189 - mean_squared_error: 0.0189 - val_loss: 0.0285 - val_mean_squared_error: 0.0285
Epoch 65/80
105/105 [==============================] - 20s 190ms/step - loss: 0.0156 - mean_squared_error: 0.0156 - val_loss: 0.0250 - val_mean_squared_error: 0.0250
Epoch 66/80
 55/105 [==============>...............] - ETA: 8s - loss: 0.0117 - mean_squared_error: 0.0117
Epoch 00066: saving model to training/cp-0066.ckpt
105/105 [==============================] - 21s 196ms/step - loss: 0.0125 - mean_squared_error: 0.0125 - val_loss: 0.0333 - val_mean_squared_error: 0.0333
Epoch 67/80
105/105 [==============================] - 20s 189ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - val_loss: 0.0288 - val_mean_squared_error: 0.0288
Epoch 68/80
  5/105 [>......................

In [ ]:
print(model.predict(x_test[10:15]))
print(y_test[10:15])

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test,y_test, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
## Save the model as a tflite version

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
## save the model regularly
model.save('/home/sorozco0612/dev/alan_bot/model_creation/alan_bot_model')